In [1]:
import numpy as np
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
prescriptions = pd.read_parquet("../20_intermediate_files/prescriptions_wa.parquet", engine = "fastparquet")

In [3]:
prescriptions.columns

Index(['BUYER_STATE', 'BUYER_COUNTY', 'CountyFIPS_x', 'StateFIPS_x', 'Year',
       'MME', 'FIP_unique', 'County', 'State', 'Population', 'county_test',
       'state_abbrev', 'CountyFIPS_y', 'StateFIPS_y', 'CountyName', '_merge'],
      dtype='object')

In [4]:
prescriptions["State"].value_counts()

NC    600
CO    372
WA    234
MD    144
Name: State, dtype: int64

In [5]:
prescriptions["Year"].value_counts()

2009.0    225
2010.0    225
2011.0    225
2012.0    225
2013.0    225
2014.0    225
Name: Year, dtype: int64

In [6]:
# creating a copy of reduced dataset of prescriptions and converting some of the attributes to appropriate data type

prescriptions_reduced_copy = prescriptions.copy()

prescriptions_reduced_copy["Year"] = prescriptions_reduced_copy["Year"].astype("int64")
# prescriptions_reduced_copy["DRUG_CODE"] = prescriptions_reduced_copy[
#     "DRUG_CODE"
# ].astype("int64")
# prescriptions_reduced_copy["Month"] = prescriptions_reduced_copy["Month"].astype(
#     "int64"
# )
# prescriptions_reduced_copy["Population"] = prescriptions_reduced_copy[
#     "Population"
# ].astype("int64")

In [7]:
prescriptions_reduced_copy.head()

,BUYER_STATE,BUYER_COUNTY,CountyFIPS_x,StateFIPS_x,Year,MME,FIP_unique,County,State,Population,county_test,state_abbrev,CountyFIPS_y,StateFIPS_y,CountyName,_merge
index,,,,,,,,,,,,,,,,
3,CO,adams,8001,8,2009,110355.414760,80018,Adams County,CO,435700.0,Adams,CO,8001.0,8.0,Adams,both
4,CO,adams,8001,8,2010,132304.866282,80018,Adams County,CO,443691.0,Adams,CO,8001.0,8.0,Adams,both
5,CO,adams,8001,8,2011,157909.630983,80018,Adams County,CO,452201.0,Adams,CO,8001.0,8.0,Adams,both
6,CO,adams,8001,8,2012,161238.256294,80018,Adams County,CO,460558.0,Adams,CO,8001.0,8.0,Adams,both
7,CO,adams,8001,8,2013,134536.857641,80018,Adams County,CO,469978.0,Adams,CO,8001.0,8.0,Adams,both


In [8]:
# creating a dataset that has all the drug prescriptions in the state of Washington

washington_prescriptions = prescriptions_reduced_copy[
    prescriptions_reduced_copy["State"] == "WA"
]

washington_prescriptions.head()

,BUYER_STATE,BUYER_COUNTY,CountyFIPS_x,StateFIPS_x,Year,MME,FIP_unique,County,State,Population,county_test,state_abbrev,CountyFIPS_y,StateFIPS_y,CountyName,_merge
index,,,,,,,,,,,,,,,,
3568,WA,adams,53001,53,2009,7673.547900,5300153,Adams County,WA,18405.0,Adams,WA,53001.0,53.0,Adams,both
3569,WA,adams,53001,53,2010,8516.658075,5300153,Adams County,WA,18790.0,Adams,WA,53001.0,53.0,Adams,both
3570,WA,adams,53001,53,2011,9310.998375,5300153,Adams County,WA,18877.0,Adams,WA,53001.0,53.0,Adams,both
3571,WA,adams,53001,53,2012,9063.213195,5300153,Adams County,WA,18944.0,Adams,WA,53001.0,53.0,Adams,both
3572,WA,adams,53001,53,2013,8987.487300,5300153,Adams County,WA,19098.0,Adams,WA,53001.0,53.0,Adams,both


In [9]:
washington_prescriptions["shipment_per_100k"] = (
        washington_prescriptions["MME"]
    / (washington_prescriptions["Population"])
    * 100_000
)

washington_prescriptions

/var/folders/gr/gh3wfyb105ddr4t6thtzf7nh0000gn/T/ipykernel_6459/97174333.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  washington_prescriptions["shipment_per_100k"] = (


,BUYER_STATE,BUYER_COUNTY,CountyFIPS_x,StateFIPS_x,Year,MME,FIP_unique,County,State,Population,county_test,state_abbrev,CountyFIPS_y,StateFIPS_y,CountyName,_merge,shipment_per_100k
index,,,,,,,,,,,,,,,,,
3568,WA,adams,53001,53,2009,7673.547900,5300153,Adams County,WA,18405.0,Adams,WA,53001.0,53.0,Adams,both,41692.735126
3569,WA,adams,53001,53,2010,8516.658075,5300153,Adams County,WA,18790.0,Adams,WA,53001.0,53.0,Adams,both,45325.482038
3570,WA,adams,53001,53,2011,9310.998375,5300153,Adams County,WA,18877.0,Adams,WA,53001.0,53.0,Adams,both,49324.566271
3571,WA,adams,53001,53,2012,9063.213195,5300153,Adams County,WA,18944.0,Adams,WA,53001.0,53.0,Adams,both,47842.130463
3572,WA,adams,53001,53,2013,8987.487300,5300153,Adams County,WA,19098.0,Adams,WA,53001.0,53.0,Adams,both,47059.835061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3911,WA,yakima,53077,53,2010,134937.738431,5307753,Yakima County,WA,244256.0,Yakima,WA,53077.0,53.0,Yakima,both,55244.390488
3912,WA,yakima,53077,53,2011,152014.707558,5307753,Yakima County,WA,245926.0,Yakima,WA,53077.0,53.0,Yakima,both,61813.190780
3913,WA,yakima,53077,53,2012,160465.293363,5307753,Yakima County,WA,246127.0,Yakima,WA,53077.0,53.0,Yakima,both,65196.135882


In [10]:
# calculating and displaying total number of drug prescriptions that took place in washington. results are grouped and displayed year and county wise 

wa_result = washington_prescriptions.groupby(["Year"])["shipment_per_100k"].mean().reset_index()
# washington_prescriptions_result = washington_prescriptions.groupby(["Year"])["shipment_per_100k"].sum().reset_index()

wa_result.head(10)

,Year,shipment_per_100k
0,2009,75058.203038
1,2010,77468.976409
2,2011,77897.686034
3,2012,77317.159398
4,2013,77655.943149
5,2014,77489.839007


In [11]:
# change Year to be integer 
wa_result["Year"] = wa_result["Year"].astype("int")
wa_result.dtypes

Year                   int64
shipment_per_100k    float64
dtype: object

In [12]:
# split into before and after policy implementation 
wa_b4 = wa_result[wa_result["Year"] < 2012]
wa_after = wa_result[wa_result["Year"] >= 2012]

In [13]:
x = "Year"
y = "shipment_per_100k"

In [14]:

# function for adding a vertical line
def vertical_line(year):
    """Function to plot a vertical line at year of policy implementation"""
    line = alt.Chart(pd.DataFrame({
    'Date': [year],
    'color': ["black"]
    })).mark_rule().encode(
    x='Date:Q',
    color=alt.Color('color:N', scale=None)
    )

    return line

In [15]:
# function for adding confidence intervals 
def get_reg_fit(data, yvar, xvar, color, title, alpha=0.05):
    import statsmodels.formula.api as smf

    # Grid for predicted values
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})

    # Fit model, get predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    reg = (
        alt.Chart(predictions)
        .mark_line()
        .encode(
            x=alt.X(xvar, scale=alt.Scale(zero=False), title="Year"),
            y=alt.Y(
                yvar, scale=alt.Scale(zero=False), title="Shipments per 100k Residents"
            ),
            color=alt.value(color)
        ).properties(title=title)
    )
    ci = (
        alt.Chart(predictions)
        .mark_errorband()
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=""),
            y2="ci_high",
            color=alt.value(color),
        )
    )
    chart = ci + reg
    return predictions, chart

In [16]:
# add a cutoff year when policy was implemented 
line = vertical_line(2012)

In [17]:
# plotting - need to make x axis discrete 
fit_wa_b4, reg_chart_wa_b4 = get_reg_fit(
    wa_b4, yvar="shipment_per_100k", xvar="Year", color="blue", title= "Shipments per 100k Residents - Washington", alpha=0.05
)


fit_wa_b4, reg_chart_wa_after = get_reg_fit(
    wa_after, yvar="shipment_per_100k", xvar="Year", color="blue", title= "Shipments per 100k Residents - Washington",alpha=0.05
)
(reg_chart_wa_b4 + reg_chart_wa_after + line).properties(width=700, height=500)

alt.LayerChart(...)

In [ ]:
########################################################################################################################
# ends here for final report     

In [ ]:
from sklearn.linear_model import LinearRegression 

regressor_b4 = LinearRegression() 
regressor_after = LinearRegression()


X_b4 = np.array(wa_b4["year relative to policy"]).reshape(-1, 1)
y_b4 = np.array(wa_b4["shipment_per_100k"]).reshape(-1, 1)

X_after = np.array(wa_after["year relative to policy"]).reshape(-1, 1)
y_after = np.array(wa_after["shipment_per_100k"]).reshape(-1, 1)


regressor_b4.fit(X_b4,y_b4) 
regressor_after.fit(X_after,y_after) 


y_pred_b4 = regressor_b4.predict(X_b4)
y_pred_after = regressor_after.predict(X_b4)

In [ ]:
y_pred_b4

In [ ]:
y_pred_after

In [ ]:
# plt.xlim(-3, 3)
# #plt.ylim(0, 500)

# plt.title("Opioid Shipments in Washington")

# plt.plot(X_b4, y_pred_b4,color='k')
# plt.plot(X_after, y_pred_after,color='k')

# # plot avg value in each year

In [ ]:
x = "Year"
y = "shipment_per_100k"

In [ ]:
def vertical_line(year):
    """Function to plot a vertical line at year of policy implementation"""
    line = alt.Chart(pd.DataFrame({
    'Date': [year],
    'color': ["black"]
    })).mark_rule().encode(
    x='Date:Q', # use q for "quantitative" - as per altair docs
    color=alt.Color('color:N', scale=None)
    )

    return line

In [ ]:
# test function
line = vertical_line(2004)

In [ ]:
def get_charts(b4, after, title_b4, title_after):
    """
    Function to plot the pre and post charts.
    Will not use in final plot - used as a baseline for our fit charts later.

    """

    base_before = (
        alt.Chart(b4)
        .mark_point()
        .encode(
            y=alt.Y("shipment_per_100k", scale=alt.Scale(zero=False)),
            x=alt.X("year relative to policy", scale=alt.Scale(zero=False)),
        )
        .properties(title=title_b4)
        
    )

    base_after = (
        alt.Chart(after)
        .mark_point()
        .encode(
            y=alt.Y("shipment_per_100k", scale=alt.Scale(zero=False)),
            x=alt.X("year relative to policy", scale=alt.Scale(zero=False)),
        )
        .properties(title=title_after)
    )

    return base_before, base_after

In [ ]:
# test the function
# may remove title parameters later - not really necessary as we aren't plotting this part in our final analysis
# however, if we can't add a title to our fit/regression line charts, we may need to add them here
base_before, base_after = get_charts(b4 = wa_b4, after = wa_after, title_b4 = "shipments before policy", title_after="shipments after policy")
base_before + base_after

In [ ]:
# starting here in final report
# no longer calculating base chart above - just adding in regression line at same time
def get_preds(df, x, y):

    # init new empty df for our predictions
    predictions = pd.DataFrame()

    # fit our model and predict values
    model = smf.ols(f"{y} ~ {x}", data=df).fit()
    model_predict = model.get_prediction(df[x])

    # save predictions back to df, calculate confidence intervals
    predictions["shipment_per_100k"] = model.predict(df[x])
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=0.05)

    # save original year columns to new predictions df
    predictions["Year"] = df["Year"]
    predictions["year relative to policy"] = df["year relative to policy"]
    return predictions

In [ ]:
wa_b4_preds = get_preds(wa_b4, x, y)
wa_after_preds = get_preds(wa_after, x, y)

In [ ]:
def get_charts(b4, after, title_b4, title_after, color):
    """
    Function to plot the pre and post charts.
    Will not use in final plot - used as a baseline for our fit charts later.

    """

    base_before = (
        alt.Chart(b4)
        .mark_point()
        .encode(
            y=alt.Y("shipment_per_100k", scale=alt.Scale(zero=False)),
            x=alt.X("year relative to policy", scale=alt.Scale(zero=False)),
        )
        .properties(title=title_b4).transform_regression("year relative to policy", "shipment_per_100k")
    .mark_line()
    .encode(color=alt.value(color))
        
    )

    base_after = (
        alt.Chart(after)
        .mark_point()
        .encode(
            y=alt.Y("shipment_per_100k", scale=alt.Scale(zero=False)),
            x=alt.X("year relative to policy", scale=alt.Scale(zero=False)),
            
        )
        .properties(title=title_after).transform_regression("year relative to policy", "shipment_per_100k")
    .mark_line()
    .encode(color=alt.value(color))
    )

    return base_before, base_after

In [ ]:
# test the function
# may remove title parameters later - not really necessary as we aren't plotting this part in our final analysis
# however, if we can't add a title to our fit/regression line charts, we may need to add them here
base_before, base_after = get_charts(b4 = wa_b4_preds, after = wa_after_preds, title_b4 = "Opioid Shipments Before and After Policy Implementation in Washington", title_after="Shipments After Policy Implementation", color="red")
base_before + base_after

In [ ]:
#####################################
# updated code for final report ends here 

In [ ]:
prescriptions["QUANTITY"].describe()

In [ ]:
prescriptions_reduced = prescriptions[['QUANTITY', 'Year', 'StateFIPS', 'StateName', 'CountyFIPS', 'FIP_unique', 'Population','county_test']]

In [ ]:
prescriptions_reduced["StateName"].value_counts()

In [ ]:
washington = prescriptions_reduced[prescriptions_reduced["StateName"] == "Washington"]
comp = prescriptions_reduced[prescriptions_reduced["StateName"] != "Washington"]

In [ ]:
wa_prescriptions = washington.copy()
comp_prescriptions = comp.copy()

In [ ]:
wa_prescriptions["quantity_sum"] = wa_prescriptions.groupby(["Year", "FIP_unique"])["QUANTITY"].transform('sum')
comp_prescriptions["quantity_sum"] = comp_prescriptions.groupby(["Year", "FIP_unique"])["QUANTITY"].transform('sum')

In [ ]:
wa_prescriptions = wa_prescriptions.drop('QUANTITY', axis=1)
comp_prescriptions = comp_prescriptions.drop('QUANTITY', axis=1)

In [ ]:
wa_result = wa_prescriptions.drop_duplicates()
comp_result = comp_prescriptions.drop_duplicates()

In [ ]:
wa_result["shipment_per_100k"] = wa_result["quantity_sum"] / wa_result["Population"] * 100_000
comp_result["shipment_per_100k"] = comp_result["quantity_sum"] / comp_result["Population"] * 100_000

In [ ]:
wa_result.groupby("Year")["shipment_per_100k"].agg([np.mean, np.std])

In [ ]:
wa_result_b4 = wa_result[wa_result["Year"] < 2012]
wa_result_after = wa_result[wa_result["Year"] >= 2012]

In [ ]:

# washington

source_data = wa_result_b4

plot_wa_b4 = alt.Chart(source_data).mark_point().encode(
    y=alt.Y("mean_shipment:Q", scale=alt.Scale(zero=False)),
    x=alt.X("Year:O", scale=alt.Scale(zero=False))
).transform_aggregate(
    mean_shipment='mean(shipment_per_100k)',
    groupby=["Year"]
)

plot_wa_b4

In [ ]:
fit_wa_b4 = plot_wa_b4.transform_regression('Year', 'mean_shipment',method="linear"
).mark_line()

fit_wa_b4

In [ ]:
source_data = wa_result_after

plot_wa_after = alt.Chart(source_data).mark_point().encode(
    y=alt.Y("mean_shipment:Q", scale=alt.Scale(zero=False)),
    x=alt.X("Year:O", scale=alt.Scale(zero=False))
).transform_aggregate(
    mean_shipment='mean(shipment_per_100k)',
    groupby=["Year"]
)

plot_wa_after

In [ ]:
fit_wa_after = plot_wa_after.transform_regression('Year', 'mean_overdose',method="linear"
).mark_line()

fit_wa_after

In [ ]:
plot_wa_b4 + fit_wa_b4 + plot_wa_after + fit_wa_after